In this notebook I walk through the generation/inferece pipeline (spending most of the time on inference).

In [1]:
from abc import ABC, abstractmethod

import os
from tqdm import tqdm
import math 
import time

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from scipy import stats

import torch
from torch.distributions import Beta
from torch.distributions.bernoulli import Bernoulli
from torch.nn.functional import log_softmax
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader, TensorDataset

In [2]:
SEED = 1
np.random.seed(SEED)
torch.manual_seed(SEED)

### Generation:

Here I am using 2/10 generation.

In [3]:
### these params control the generation scheme
rho        = 0.8   # polarization
pop_size   = 50000 # num individuals
epsilon    = 0.05  # expected prop of speech consisting of neutral words
pi         = 0.5   # pi == 0.5 => beta mixture symmetrical (choose beta1 with prob pi = 0.5)
speech_len = 15    # words per speech

In [7]:
def generate(rho=rho, N=pop_size, epsilon=epsilon, pi=pi, speech_len=speech_len):
    """
    Uses 2/10 generation scheme to generate N samples.

    Returns:
        (X, y), (a, b, rho, epsilon, u)
        X.size() == [N, 3] is a vector of word counts
        y.size() == [N] is a vector of political parties
        u.shape  == (N,) is a vector of individual stances
        rho is true polarization
        epsilon is expected prop of neutral words
        a, b are true alpha/beta for beta mixture model
    """
    start = time.time()
    print(f'Beginning Data Generation...')
    print(f'=' * 20)
    
    ### get beta mixture model params 
    sigma = 0.175 * (rho ** 2) - 0.3625 * rho + 0.1875
    a     = rho * ((rho * (1 - rho)) / sigma - 1)
    b     = (1 - rho) * ((rho * (1 - rho)) / sigma - 1)

    print(f"True Alpha: {a}")
    print(f"True Beta: {b}")
    print(f"True Epsilon: {epsilon}\n")

    mean = a / (a + b)
    var  = a * b / ((a + b)**2 * (a + b + 1))

    if abs(mean - rho) > 10e-15:
        print(f'Mean: {mean}')
        print(f"Rho: {rho}")
        raise AssertionError(f"Mean of BMM params should be rho")

    if abs(var - sigma) > 10e-15:
        print(f'Var: {var}')
        print(f'Sigma: {sigma}')
        raise AssertionError(f"Var of BMM params should be sigma")

    ### u ~ pi Beta(a, b) + (1 - pi) Beta(b, a)
    weights = [pi, 1-pi]
    mixture_samples = np.random.choice([0, 1], size=N, p=weights)

    u = 2 * np.where(mixture_samples == 1, stats.beta.rvs(a, b, size=N), stats.beta.rvs(b, a, size=N)) - 1

    if u.shape != (N,):
        raise AssertionError(f"u.shape should be (N,)")

    print(f'mixture samples: {mixture_samples[:5]}')
    print(f'u samples: {u[:5]}\n')

    ### y = 1(u >= 0)
    y = (u >= 0).astype(int)
    print(f'y samples: {y[:5]}\n')

    ### phi is a prob matrix that is a function of u, epsilon
    phi = np.array([(1 - (u+1)/2) * (1 - epsilon), (u+1)/2 * (1 - epsilon), np.repeat(epsilon, pop_size)]).T
    print(f'phi samples:\n {phi[:5, :]}\n')

    if phi.shape != (N, 3):
        raise AssertionError(f'phi.shape should be (N, V) == (N, 3)')

    if abs(sum(phi[0]) - 1) > 10e-5:
        raise AssertionError(f'rows of phi should sum to 1')
    
    X = np.array([stats.multinomial.rvs(n=speech_len, p=phi[i, :]) for i in range(N)])
    print(f'X samples:\n {X[:5, :]}\n')
    
    if X.shape != (N, 3):
        raise AssertionError(f'X.shape should be (N, V) == (N, 3)')

    if X[:5].sum() != 15 * 5:
        raise AssertionError(f'rows of phi should sum to 1')

    X = torch.from_numpy(X).to(torch.float32)
    y = torch.from_numpy(y).to(torch.float32)

    known   = (X, y)
    unknown = (a, b, rho, epsilon, u)
    print('=' * 20)
    print(f'Generation Time: {round(time.time() - start, 3)} seconds for {N} samples.')
    return known, unknown

known, unknown = generate()
X, y = known
a, b, rho, epsilon, u = unknown

Beginning Data Generation...
True Alpha: 12.673684210526261
True Beta: 3.1684210526315644
True Epsilon: 0.05

mixture samples: [1 0 1 1 1]
u samples: [ 0.58836654 -0.44066272  0.36309715  0.47382017  0.55063803]

y samples: [1 0 1 1 1]

phi samples:
 [[0.19552589 0.75447411 0.05      ]
 [0.68431479 0.26568521 0.05      ]
 [0.30252886 0.64747114 0.05      ]
 [0.24993542 0.70006458 0.05      ]
 [0.21344694 0.73655306 0.05      ]]

X samples:
 [[ 5  9  1]
 [12  3  0]
 [ 7  6  2]
 [ 2 12  1]
 [ 4 11  0]]

Generation Time: 1.637 seconds for 50000 samples.


### Inference

The goal of inference is to recover $u$ given a sample of $(x,y)$ pairs. We assume the following data generating process:
\begin{align*}
    u &\sim 2 \cdot \left(\pi\cdot\mathrm{Beta}\left(u;\alpha,\beta\right)+(1-\pi)\cdot\mathrm{Beta}\left(u;\beta,\alpha\right)\right)-1 \\
    y &= 1(u \ge \lambda) \\
    x &\sim \mathrm{Multinomial}\left(S, \mathrm{Softmax}(Wu)\right), W \in \R^V.
\end{align*}

For now, we assume $\lambda = 0, V = 3, \pi = 0.5$, so our parameters are $\theta = \{\alpha, \beta, W\}$. Given we assume this full form of $u$ parameterized by $\alpha, \beta$, recovering the distribution of $u$ is equivalent to recovering $\alpha, \beta$. This also gives us the following joint:

\begin{align*}

p(u,x^{(n)},y^{(n)};\theta) &= p(u)p(y^{(n)}|u)\prod_{s=1}^{S}p(x^{(n)}_s | u) \\
&= \left(\frac{1}{4}\mathrm{Beta}\left(\frac{u+1}{2};\alpha,\beta\right)+\frac{1}{4}\mathrm{Beta}\left(\frac{u+1}{2};\beta,\alpha\right)\right) \\&\cdot 1\left(1(u\ge0)=y^{(n)}\right)\frac{S!}{x^{(n)}_1!x^{(n)}_2!x^{(n)}_3!}\prod_{s=1}^{S}\mathrm{softmax}(Wu)_{x^{(n)}_s}

\end{align*}

Here, we can get the density of $u$ by noting $u$ is a monotonic transform of a random variable whose density is known. Thus, applying a formula found [here](https://en.wikipedia.org/wiki/Probability_density_function#Function_of_random_variables_and_change_of_variables_in_the_probability_density_function) we get the above expression.

The basic idea is to maximize the log marginal likelihood. Let $\theta = \{\alpha, \beta, W\}$. We'd like to find 
$$
\theta^* = \argmax_{\theta} \underbrace{\log p(x^{(1:N)}, y^{(1:N)}; \theta)}_{\mathcal{L}(\theta)}. 
$$


The goal is to recover $\alpha, \beta$ (hence $u$ given we are assuming $u \sim 2\left(\frac{1}{2} \cdot \mathrm{Beta}(\alpha, \beta) + \frac{1}{2} \cdot \mathrm{Beta}(\beta, \alpha)\right) - 1$ given $(x,y)$ pairs.

To get MLE estimates for alpha, beta, W, we will take gradient descent steps on the log marginal likelihood. Its gradient is given by the following expression:
$$
\nabla_\theta \log p(x^{(1:N)},y^{(1:N)}) = \sum_{n=1}^{N} \mathbb{E}_{p(u|x^{(n)},y^{(n)};\theta)}[\nabla_\theta \log p(u,x^{(n)},y^{(n)};\theta)].
$$
There are a number of issues here, beginning with the fact that we cannot analytically recover the posterior due to the integral being intractable. However, since $u \in [-1,1]$, we can discretize the integral, which will be simpler than something like MCMC for sampling/using variational inference to approximate the posterior. To do this, let's write the gradient as follows:
$$
\nabla_\theta \log p(x^{(1:N)},y^{(1:N)}) = \sum_{n=1}^{N} \int_{u} \frac{p(x^{(n)}, y^{(n)}, u;\ \theta)}{p(x^{(n)},y^{(n)};\  \theta)}\nabla_\theta \log p(u,x^{(n)},y^{(n)};\theta) \mathrm{d}u
$$
To discretize, we simply need to assume that $u$ takes on $G$ linearly spaced points in $[-1,1]$, rather than is continuous in $[-1,1]$, which gives us 
$$
\nabla_\theta \log p(x^{(1:N)},y^{(1:N)}) = \sum_{n=1}^{N} \sum_{u} \frac{p(x^{(n)}, y^{(n)}, u;\ \theta)}{p(x^{(n)},y^{(n)};\  \theta)}\nabla_\theta \log p(u,x^{(n)},y^{(n)};\theta)
$$
Notice that when $y \ne 1(u \ge 0)$, the joint probability $p(x^{(n)}, y^{(n)}, u;\ \theta)$ is $0$ ($u$ and $y$ are 'incompatible'). Thus, in practice we can use $y^{(n)}$ as a condition: if $y^{(n)} = 1$ just compute $p(u, x^{(n)}, y^{(n)})$ for $G$ linearly spaced points in $[0,1]$, and then normalize that probability.

It's convenient to move the gradient outside the sum, since then we can efficiently compute the double sum via matrices and then just sum the matrix/call .backward(). This is what I was doing initially, but it's definitely wrong, since the posterior relies on theta. To fix this, I call .detach() on the "weight" (posterior probability), which means the gradients only flow through the joint log probability (which is what we want).

In [31]:
grid_size  = 200 ### discretize integral into grid_size linearly spaced pts
batch_size = 100 ### num samples processed simultaneously
num_epochs = 200 ### number epochs
lr         = 0.1 ### learning rate

def compute_nll(x_batch, y_batch, log_alpha, log_beta, W):
    """
    Parameters:
        (x_batch, y_batch) a batch of B (x,y) samples 
        (log_alpha, log_beta, W) trainable parameters

    Returns:
        A scaler quantity s.t. calling .backward() will compute
        grad_{theta} (L(theta))
    """
    raise NotImplementedError

def train(X, y, num_epochs=num_epochs, batch_size=batch_size, grid_size=grid_size, lr=lr):
    print(f'Beginning Inference...')
    print(f'=' * 20)
    print(f'Hyperparameters:')
    print(f'Dataset Length: {len(X)}')
    print(f'Number Epochs: {num_epochs}')
    print(f'Batch Size: {batch_size}')
    print(f'Grid Size: {grid_size}')
    print(f"Learning Rate: {lr}")
    print('=' * 20)
    
    start = time.time()

    dataset    = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    X_batch, y_batch = list(dataloader)[0]

    if list(X_batch.size()) != [batch_size, 3]:
        raise AssertionError(f"X_ex.size() should be [batch_size, V] == [batch_size, 3], got {X_batch.size()}")

    if list(y_batch.size()) != [batch_size]:
        raise AssertionError(f"y_ex.size() should be [batch_size], got {y_batch.size()}")

    ### play with parameter initialization!
    log_alpha = torch.normal(1.5, 1, size=(1,), requires_grad=True)
    log_beta  = torch.normal(0, 1, size=(1,), requires_grad=True)
    W         = torch.normal(3, 2, size=(3,), requires_grad=True)

    print(f'Initial Parameters:')
    print(f'Alpha: {torch.exp(log_alpha).item()}')
    print(f"Beta: {torch.exp(log_beta).item()}")
    print(f"W: {W.data.tolist()}")
    optimizer = SGD([log_alpha, log_beta, W], lr=lr)
    
    for epoch in tqdm(range(num_epochs)):
        print(f'Epoch {epoch+1} of {num_epochs}')
        ### only step after each epoch
        optimizer.zero_grad() 
        nll = 0
        for x_batch, y_batch in dataloader:
            nll += compute_nll(x_batch, y_batch, log_alpha, log_beta, W, grid_size)
        
        nll = nll / len(dataloader.dataset)
        nll.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, NLL: {round(nll.item(), 3)}, Alpha: {round(np.exp(log_alpha.item()),3)}, Beta: {round(np.exp(log_beta.item()),3)}, W: {W.tolist()}')
        print('=' * 20)

    final_alpha = np.exp(log_alpha.item())
    final_beta  = np.exp(log_beta.item())

    print(f"Training Took {round(time.time() - start, 2)} Seconds.")
    print(f"Trained Params: alpha: {final_alpha}, beta: {final_beta}, W: {W.tolist()}")

    return final_alpha, final_beta, W

In [32]:
factorial = lambda x : torch.exp(torch.lgamma(x+1))

def compute_log_joint(u_mat, x_batch, y_batch, log_alpha, log_beta, W):
    """
    Parameters:
        x_batch (size [batch_size, 3])
        y_batch (size [batch_size])
        u_mat   (size [batch_size, N])
        (log_alpha, log_beta, W) trainable params 
        constants (bool) true iff include constants
    
    constants are necessary for computing joint probabilities exactly but not for gradient expressions
    
    Returns:
        torch.tensor (size [batch_size, N]), where the (i,j)th element 
        is the log joint probability (log p(u, x, y; theta)). Here u is the (i,j)th
        element of u_mat, x is the ith row of x_batch, y is the ith element of y_batch

    """
    ### log prior: log p(u) = log(1/4 Beta((u+1)/2; a, b) + 1/4 Beta((u+1)/2; b, a))
    alpha = torch.exp(log_alpha)
    beta  = torch.exp(log_beta)
        
    beta_dist_ab = Beta(alpha, beta)
    beta_dist_ba = Beta(beta, alpha)

    log_beta_prob_ab = beta_dist_ab.log_prob((u_mat + 1) / 2)
    log_beta_prob_ba = beta_dist_ba.log_prob((u_mat + 1) / 2)

    prior = torch.log(torch.exp(log_beta_prob_ab) + torch.exp(log_beta_prob_ba)) + torch.log(torch.tensor(0.25))

    assert prior.size() == u_mat.size()
  
    ### log likelihood: log p(x, y | u) = dot(x, log Softmax(Wu)) + log(S!) - log(x1!) - log(x2!) - log(x3!) (assuming y is compatible)
    W_expanded = W.unsqueeze(0).unsqueeze(0)
    assert list(W_expanded.size()) == [1,1,3]
    u_expanded = u_mat.unsqueeze(2)

    softmax_input = torch.matmul(u_expanded, W_expanded)  ### size BxNx3
    likelihood = (x_batch.unsqueeze(1) * log_softmax(softmax_input, dim=2)).sum(dim=2) ### size BxN
    
    likelihood += torch.log(factorial(torch.tensor(speech_len)))  ### log(S!)
    likelihood -= torch.log(factorial(x_batch)).sum(dim=1, keepdim=True) # - log(x_1!) -log(x_2!) - log(x_3!)

    return prior + likelihood


def compute_nll(x_batch, y_batch, log_alpha, log_beta, W, grid_size):
    """
    Parameters:
        x_batch (size [batch_size, 3])
        y_batch (size [batch_size])
    
    Returns:
        torch.tensor (size [batch_size, approx_size]), where the ith row are 
        samples from the approximate posterior p(u | x^(i), y^(i); theta).
        Here, x^(i), y^(i) are the ith row of x_batch/y_batch. 
    """
    u_mat = torch.empty(batch_size, grid_size)
    u_mat[y_batch == 1] = torch.linspace(1/(grid_size+1), 1-1/(grid_size+1), grid_size).repeat((y_batch == 1).sum(), 1)
    u_mat[y_batch == 0] = torch.linspace(-1+1/(grid_size+1), -1/(grid_size+1), grid_size).repeat((y_batch == 0).sum(), 1)

    assert list(u_mat.size()) == [x_batch.size()[0], grid_size]

    log_joint = compute_log_joint(u_mat, x_batch, y_batch, log_alpha, log_beta, W)

    ### log sum exp (don't need to add back bc scaling)
    max_log_prob = torch.max(log_joint, dim=1, keepdim=True)[0]
    joint_probs = torch.exp(log_joint - max_log_prob)
    posterior = joint_probs / joint_probs.sum(dim=1, keepdim=True)  

    # detach posterior
    weighted_log_joint = posterior.detach() * log_joint
    nll = -weighted_log_joint.sum()

    return nll

In [33]:
train(X, y)

Beginning Inference...
Hyperparameters:
Dataset Length: 50000
Number Epochs: 200
Batch Size: 100
Grid Size: 200
Learning Rate: 0.1
Initial Parameters:
Alpha: 2.9923245906829834
Beta: 5.803126811981201
W: [2.1709372997283936, 4.712274551391602, 2.114123821258545]


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1 of 200


  0%|          | 1/200 [00:06<20:48,  6.27s/it]

Epoch 1, NLL: 6.908, Alpha: 2.825, Beta: 6.041, W: [2.0754234790802, 4.7453532218933105, 2.1765589714050293]
Epoch 2 of 200


...

100%|█████████▉| 199/200 [14:30<00:05,  5.30s/it]

Epoch 199, NLL: 5.567, Alpha: 2.047, Beta: 6.259, W: [0.9280070066452026, 5.070655345916748, 2.998678684234619]
Epoch 200 of 200


100%|██████████| 200/200 [14:39<00:00,  4.40s/it]

Epoch 200, NLL: 5.566, Alpha: 2.048, Beta: 6.258, W: [0.9274062514305115, 5.071256637573242, 2.998678207397461]
Training Took 879.89 Seconds.
Trained Params: alpha: 2.047592904495089, beta: 6.257558317257251, W: [0.9274062514305115, 5.071256637573242, 2.998678207397461]


(2.047592904495089,
 6.257558317257251,
 tensor([0.9274, 5.0713, 2.9987], requires_grad=True))

In [35]:
6.257558317257251/(6.257558317257251 + 2.047592904495089)

0.7534550726623545